In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Menyiapkan Data

In [ ]:
df = pd.read_csv('/content/Hoax_training.csv')
df.head()

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


# Preprocessing

## Case Folding

In [ ]:
# Mengubah semua huruf pada pada kolom judul dan narasi menjadi huruf kecil

df['judul'] = df['judul'].str.lower()
df['narasi'] = df['narasi'].str.lower()

In [ ]:
df

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,pemakaian masker menyebabkan penyakit legionna...,a caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,instruksi gubernur jateng tentang penilangan ...,yth.seluruh anggota grup sesuai instruksi gube...,461.png
2,495,1,13-Jul-20,foto jim rohn: jokowi adalah presiden terbaik ...,jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan pak jokowi b...","maaf mas2 dan mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,foto kadrun kalo lihat foto ini panas dingin,kadrun kalo lihat foto ini panas dingin . .,681.jpg
...,...,...,...,...,...,...
4226,999287,1,17-Jan-16,: kpk dilarang membawa brimob bersenjata masuk...,kpk dilarang membawa brimob bersenjata masuk g...,999287.jpg
4227,999472,1,20-Aug-20,foto pejabat keuangan dibawah palu arit,jangan mau dialihkan kepada pakaian adat… foku...,999472.png
4228,999501,1,17-Aug-20,gambar denny siregar musuh warga tasikmalaya d...,akhir nya bisa terkenal bang denny,999501.jpg
4229,999848,1,21-Jul-20,kaesang: bapak saya dengan kesederhaan bisa ni...,bapak saya dengan kesederhaan bisa nipu rakyat...,999848.jpg


# Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

df['judul'] = df['judul'].apply(stemmed_wrapper)

In [ ]:
df

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,pakai masker sebab sakit legionnaires,a caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,instruksi gubernur jateng tentang tilang bagi ...,yth.seluruh anggota grup sesuai instruksi gube...,461.png
2,495,1,13-Jul-20,foto jim rohn jokowi adalah presiden baik dlm ...,jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,ini bukan politik tapi nyata pak jokowi hasil ...,"maaf mas2 dan mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,foto kadrun kalo lihat foto ini panas dingin,kadrun kalo lihat foto ini panas dingin . .,681.jpg
...,...,...,...,...,...,...
4226,999287,1,17-Jan-16,kpk larang bawa brimob senjata masuk gedung dpr,kpk dilarang membawa brimob bersenjata masuk g...,999287.jpg
4227,999472,1,20-Aug-20,foto jabat uang bawah palu arit,jangan mau dialihkan kepada pakaian adat… foku...,999472.png
4228,999501,1,17-Aug-20,gambar denny siregar musuh warga tasikmalaya d...,akhir nya bisa terkenal bang denny,999501.jpg
4229,999848,1,21-Jul-20,kaesang bapak saya dengan kesederhaan bisa nip...,bapak saya dengan kesederhaan bisa nipu rakyat...,999848.jpg


# Tokenizing

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
import string 
import re

# Hapus angka
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['judul'] = df['judul'].apply(remove_number)
df['narasi'] = df['narasi'].apply(remove_number)

# Hapus tanda baca
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['judul'] = df['judul'].apply(remove_punctuation)
df['narasi'] = df['narasi'].apply(remove_punctuation)

# NLTK tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['judul'] = df['judul'].apply(word_tokenize_wrapper)
df['narasi'] = df['narasi'].apply(word_tokenize_wrapper)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,"[pakai, masker, sebab, sakit, legionnaires]","[a, caller, to, a, radio, talk, show, recently...",71.jpg
1,461,1,17-Jul-20,"[instruksi, gubernur, jateng, tentang, tilang,...","[ythseluruh, anggota, grup, sesuai, instruksi,...",461.png
2,495,1,13-Jul-20,"[foto, jim, rohn, jokowi, adalah, presiden, ba...","[jokowi, adalah, presiden, terbaik, dlm, sejar...",495.png
3,550,1,8-Jul-20,"[ini, bukan, politik, tapi, nyata, pak, jokowi...","[maaf, mas, dan, mbak, ini, bukan, politik, ta...",550.png
4,681,1,24-Jun-20,"[foto, kadrun, kalo, lihat, foto, ini, panas, ...","[kadrun, kalo, lihat, foto, ini, panas, dingin]",681.jpg
...,...,...,...,...,...,...
4226,999287,1,17-Jan-16,"[kpk, larang, bawa, brimob, senjata, masuk, ge...","[kpk, dilarang, membawa, brimob, bersenjata, m...",999287.jpg
4227,999472,1,20-Aug-20,"[foto, jabat, uang, bawah, palu, arit]","[jangan, mau, dialihkan, kepada, pakaian, adat...",999472.png
4228,999501,1,17-Aug-20,"[gambar, denny, siregar, musuh, warga, tasikma...","[akhir, nya, bisa, terkenal, bang, denny]",999501.jpg
4229,999848,1,21-Jul-20,"[kaesang, bapak, saya, dengan, kesederhaan, bi...","[bapak, saya, dengan, kesederhaan, bisa, nipu,...",999848.jpg


# Stopword Removal

In [ ]:
# pemilihan kata-kata penting
nltk.download('stopwords')

list_stopwords = stopwords.words('indonesian')

def stopwords_removal(words):
  return [word for word in words if word not in list_stopwords]

df['judul'] = df['judul'].apply(stopwords_removal) 
df['narasi'] = df['narasi'].apply(stopwords_removal) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,"[pakai, masker, sakit, legionnaires]","[a, caller, to, a, radio, talk, show, recently...",71.jpg
1,461,1,17-Jul-20,"[instruksi, gubernur, jateng, tilang, yg, mask...","[ythseluruh, anggota, grup, sesuai, instruksi,...",461.png
2,495,1,13-Jul-20,"[foto, jim, rohn, jokowi, presiden, dlm, sejar...","[jokowi, presiden, terbaik, dlm, sejarah, bang...",495.png
3,550,1,8-Jul-20,"[politik, nyata, jokowi, hasil, pulang, triliu...","[maaf, mas, mbak, politik, kenyataan, jokowi, ...",550.png
4,681,1,24-Jun-20,"[foto, kadrun, kalo, lihat, foto, panas, dingin]","[kadrun, kalo, lihat, foto, panas, dingin]",681.jpg
...,...,...,...,...,...,...
4226,999287,1,17-Jan-16,"[kpk, larang, bawa, brimob, senjata, masuk, ge...","[kpk, dilarang, membawa, brimob, bersenjata, m...",999287.jpg
4227,999472,1,20-Aug-20,"[foto, jabat, uang, palu, arit]","[dialihkan, pakaian, adat…, fokus, isyarat, co...",999472.png
4228,999501,1,17-Aug-20,"[gambar, denny, siregar, musuh, warga, tasikma...","[nya, terkenal, bang, denny]",999501.jpg
4229,999848,1,21-Jul-20,"[kaesang, kesederhaan, nipu, rakyat, indonesia]","[kesederhaan, nipu, rakyat, indonesia]",999848.jpg


#Export Data Hasil Preprocessing

In [ ]:
df.to_csv("Hoax_training_Preprocessing.csv")

# SVM

In [ ]:
import math

## bow

In [ ]:
def bowCreate(documents):
    return set(word for document in documents for word in document.split())

## frekuensi index

In [ ]:
def frekuansiIndex(documents,bow):
    return [{word:dok.count(word) for word in bow } for dok in documents ]

## VSM document

In [ ]:
def vectorSpaceModel(documents,bow):
    frekuansi = frekuansiIndex(documents,bow)
    return [[i[j] for j in i] for i in frekuansi]

## VSM query

In [ ]:
def vectorQuery(query,bow):
    return [query.count(j) for j in bow ]

## cosinus similarity

In [ ]:
def cosSimilarity(query,documents):
    bow = bowCreate(documents)
    vector_space_model = vectorSpaceModel(documents,bow)
    vector_query = vectorQuery(query,bow)
    vector_query_bottom = sum([i**2 for i in vector_query])
    result= []
    for i in range(len(vector_space_model)):
        upper =0
        bottom= 0
        for j in range(len(vector_space_model[i])):
            upper += vector_space_model[i][j]*vector_query[j]
            bottom +=vector_space_model[i][j]**2
        result_temp = upper/math.sqrt(vector_query_bottom*bottom) if vector_query_bottom!=0 | bottom !=0 else 0
        result.append({'id':i,'value':result_temp,})
    return result

## join data

In [ ]:
judul = list(map(lambda x:" ".join(x),df['judul'].values.tolist()))

In [ ]:
narasi = list(map(lambda x:" ".join(x),df['narasi'].values.tolist()))

## Execute

In [ ]:
value = cosSimilarity("jokowi",judul+narasi)

In [ ]:
value[0]['value']

0.20534249141832392

In [ ]:
sortedval = sorted(value,key=lambda x:x['value'],reverse=True)

In [ ]:
sortedval[:5]

[{'id': 5589, 'value': 0.9459053029269173},
 {'id': 955, 'value': 0.795411678668406},
 {'id': 3278, 'value': 0.7902958142673179},
 {'id': 6033, 'value': 0.7833128218667024},
 {'id': 4037, 'value': 0.7521804311928878}]

In [ ]:
narasi[5589-len(judul)]

'jokowi😭😭'

In [ ]:
judul[955]

'bocor jokowi asli'

In [ ]:
narasi[3278-len(judul)]

'kkn ugm jokowi…sembilan dosen menteri…untuk menjaga ijazah palsu sma jokowi'

In [ ]:
narasi[6033-len(judul)]

'jokowi jokowi jokowi cobong cebong cebong nih pkinya jokowi cebong'

In [ ]:
narasi[4037-len(judul)]

'“ lagilagi terbukti shalat padang pencitraan shaf sengaja dimundurkan wartawan mudah memotret adegan shalatnya'